# Working with Raster Data in Python

Python has an ecosystem of packages to work with Geospatial data. The most important packages to work with Raster data are:

- [Rasterio](https://rasterio.readthedocs.io/en/latest/index.html) is a highly useful module for raster processing which you can use for reading and writing [several different raster formats](https://gdal.org/drivers/raster/index.html) in Python. Rasterio is based on [GDAL](http://www.gdal.org/). Most common file formats that can be read with `Rasterio` include for example TIFF, GeoTIFF, ASCII Grid and Erdas Imagine (.img) files.
- [xarray](https://docs.xarray.dev/en/stable/getting-started-guide/index.html) is a package to create array objects similar to numpy arrays, but store extra information (dimensions, coordinates, coordinate reference system, time). Data model based on NetCDF. Xarray has built in functionality for parallel computing. 
- [rioxarray](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html) is basically an extension for `xarray` based on `rasterio`. With rioxarray you can read geospatial data files directly into `xarray` objects.
- [matplotlib](https://matplotlib.org/) is the most common plotting library in Python, inspired by the plotting functionalities of MATLAB
- [earthpy](https://pypi.org/project/earthpy/) provides some extra plotting functionalities specifically for geospatial datasets.


## Before we start

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<p style='margin-top:1em;margin-left:1em'>
<b>Step 1: change the Python kernel</b></p>
<p style='margin-left:1em;'>
Make sure in the top right of this window the selected kernel is `geo-kernel (ipykernel)`. If you see `Python 3 (ipykernel)`, just click on it to change it to the `geo-kernel`.
</p>
</span>
</div>


## Reading raster file (rioxarray/rasterio)

For reading data we use the `rioxarray` package. `rasterio` also has functions to read data, with this package you will get a slightly different data type that has slightly different functionalities. Basically you could do most of the steps below by using `rasterio`, but the commands will look slightly different.

In [ ]:
import rioxarray
import os
%matplotlib inline

# Data dir
data_dir = "../data/input_data/"
fp = os.path.join(data_dir, "USP_4Band_RGBN.tif")

# Open the file:
raster = rioxarray.open_rasterio(fp, band_as_variable = True)

# Check type of the variable 'raster'
type(raster)

## Exploring file structure (Dataset class)

It is always good to explore a bit the structure and content of data objects before working with it. Especially with geospatial data you want to know the dimensions, the coordinate reference system and the number of layers or bands.

In [ ]:
raster

### Extracting bands into separate objects

We will now create objects of each of the spectral bands of the image. It is not strictly necessary to do this (it is a bit inefficient to make copies of the data), but it makes the python code below better readable and intuitive.

In [ ]:
blue = raster["band_3"]
green = raster["band_2"]
red = raster["band_1"]
nir = raster["band_4"]
blue

In [ ]:
print(red.rio.crs)
print(red.rio.nodata)
print(red.rio.bounds())
print(red.rio.width)
print(red.rio.height)

You can see that the individual bands are a different data type (`xarray.DataArray` instead of `xarray.Dataset`). The `DataArray` is slightly simpler, but still contains the dimensions and coordinates of the image.

### Visualizing bands

`xarray.DataArray` s have a built-in plot method (this is based on the plotting functions from the `matplotlib` package). We can create plots for the red band by running this command:

In [ ]:
red.plot()

## Coordinate Reference Systems

To define the location of something you often use a coordinate reference system (CRS). This system consists of an X and a Y value located within a 2 (or more) -dimensional space. Because we typically make rectangular maps from an area on a (not perfectly shaped) sphere, 'we' invented an big number of CRSs to make this maps look pretty. The downside of this is that you can collect different datasets in different CRSs which can be a pain to work with. Luckily there are tools to convert a CRS to a different CRS.

A coordinate reference system is made up of several key components:

- Coordinate system: The X, Y grid upon which your data is overlayed and how you define where a point is located in space.
- Horizontal and vertical units: The units used to define the grid along the x, y (and z) axis.
- Datum: A modeled version of the shape of the Earth which defines the origin used to place the coordinate system in space. You will learn this further below.
- Projection Information: The mathematical equation used to flatten objects that are on a round surface (e.g. the Earth) so you can view them on a flat surface (e.g. your computer screens or a paper map).

Commonly used CRSs include: 

WGS 84: https://epsg.io/4326  
UTM31N: https://epsg.io/32631

## CRS conversion

Actually I find the coordinates in the plot above not really intuitive. I prefer "lat, long", so degrees latitude and longitude. Let's change the CRS using the `crs` module from the `rasterio` package.

In [ ]:
from rasterio.crs import CRS

crs_wgs84 = CRS.from_string('EPSG:4326')

In [ ]:
red_wgs84 = red.rio.reproject(crs_wgs84)

In [ ]:
red_wgs84.plot()

As you can see, the coordinates and labels have changed. We can also inspect the `DataArray` to see what changed:

In [ ]:
red_wgs84

## Cropping

Now, let's say we want to zoom in to the area where we are currently are and do some analysis on the vegetation in this area. We will first need to crop the image. It is best to do this first, it will result in a smaller image that is easier to work with. We can use the `clip_box` method to do this. We will do this for the reprojected red spectral band image first:

In [ ]:
red_clipped = red_wgs84.rio.clip_box(
    minx=5.170,
    miny=52.088,
    maxx=5.180,
    maxy=52.092,
)
red_clipped.plot()

And now there is the very convenient function `reproject_match` that will -in one go- perform the reprojection and cropping step that we now only applied to the red spectral band. Basically it will make sure the `xarray.DataArray` object it will be applied on has the same dimensions and coordinates as another object that we specify:

In [ ]:
green_clipped = green.rio.reproject_match(red_clipped)
green_clipped.plot()

Similarly we will apply this method on the blue and near infrared spectral bands:

In [ ]:
blue_clipped = blue.rio.reproject_match(red_clipped)
nir_clipped = nir.rio.reproject_match(red_clipped)

## Composing a True color (or RGB) image

We can use the red, green and blue spectral bands to create a true color image. As you can see in the legends above the values are scaled between 0 and 255. For composing the image we need to normalize the values first:

In [ ]:
# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
redn = normalize(red_clipped)
greenn = normalize(green_clipped)
bluen = normalize(blue_clipped)
nirn = normalize(nir_clipped)

print("Normalized bands")
print(redn.values.min(), '-', redn.values.max(), 'mean:', redn.values.mean())
print(greenn.values.min(), '-', greenn.values.max(), 'mean:', greenn.values.mean())
print(bluen.values.min(), '-', bluen.values.max(), 'mean:', bluen.values.mean())

Now let's merge the three bands into one (`DataArray`) object. For this we can use the `merge` function from the `xarray` package:

In [ ]:
import xarray as xr

# Create RGB natural color composite
#rgb = xr.merge([red_clipped, green_clipped, blue_clipped]).to_array()
rgb = xr.merge([redn, greenn, bluen]).to_array()

rgb

Now we can plot the true color image:

In [ ]:
rgb.plot.imshow(figsize=(8,8))

## NDVI and raster maths

It is possible to do all kinds of mathematical operations using 1 or more raster layers. We have already seen how to normalize a raster layer above. Now we will combine the red and infrared spectral bands to calculate the Normalized Difference Vegetation Index (NDVI):

$$ NDVI = \frac{NIR - red}{NIR + red} $$

This index is commonly used to quantify the amount of green leaf.

First, let's check if the shape of the rasters is the same:

In [ ]:
print(redn.shape)
print(nirn.shape)

Now we simply use the formula above where we use the bands as variables. The ndvi will be calculated on a pixel basis.

In [ ]:
ndvi = (nirn - redn)/ (nirn + redn)

In [ ]:
ndvi

In [ ]:
ndvi.plot()

Now let's save this image to a file:

In [ ]:
ndvi_nonan = ndvi.interpolate_na(dim="x")
ndvi_nonan.rio.to_raster("NDVI.tif")

## Image Classification

Using raster math and raster operations it is possible to creating new maps. In the following example we will use the NDVI map from above to create a new map where we classify NDVI values into vegetation density categories. 

Let's first create a histogram of all the values to see their distribution:




In [ ]:
ndvi.plot.hist()

Now let's subdivide the pixels into 4 classes:

In [ ]:
import numpy as np
import xarray

# Defines the bins for pixel values
class_bins = (-1, -0.2, 0.2, 0.5, 1)

# The numpy.digitize function returns an unlabeled array, in this case, a
# classified array without any metadata. That doesn't work--we need the
# coordinates and other spatial metadata. We can get around this using
# xarray.apply_ufunc, which can run the function across the data array while
# preserving metadata.
ndvi_classified = xarray.apply_ufunc(
    np.digitize,
    ndvi_nonan,
    class_bins
)

Now we want to plot the result. We will create labels for the different categories and assign them different colors.

In [ ]:
import earthpy.plot as ep
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

# Define color map of the map legend
ndvi_colors = ["blue", "gray", "green", "darkgreen"]
ndvi_cmap = ListedColormap(ndvi_colors)

# Define class names for the legend
category_names = [
    "Water",
    "No Vegetation",
    "Sparse Vegetation",
    "Dense Vegetation"
]

# We need to know in what order the legend items should be arranged
category_indices = list(range(len(category_names)))

# Make the plot
im = ndvi_classified.plot(cmap=ndvi_cmap, add_colorbar=False)
plt.title("Classified NDVI")
# earthpy helps us by drawing a legend given an existing image plot and legend items, plus indices
ep.draw_legend(im_ax=im, classes=category_indices, titles=category_names)

# Save the figure
plt.savefig("NDVI_classified.png", bbox_inches="tight", dpi=300)

## Main concluding messages:

We have done the following steps:
- Reading a GeoTIFF file (RGB image)
- Extracting the different spectral bands from the image into separate objects
- Reprojecting and converting the CRS
- Cropping
- Composing a True color image
- Raster math / NDVI
- Image classification

This is a very short demonstrations of some of the possiblities of working with Raster data in Python. Some key advantages of working with Python:
- Repeatable: Steps are easy to repeat for a new file or new selection, value, etc.
- Reproducible: Publish the script with your manuscript, so all your steps can be reproduced
- Scalable: built-in parallel options Xarray package
- Large ecosystem of (community developed) libraries to do a wide variety of other cool things
